In [14]:
#pip install python-docx

In [1]:
import os
import pickle

import zipfile
from io import BytesIO
from docx import Document

import pandas as pd
import re
from datetime import datetime

from tqdm import tqdm
# Set tqdm to work with pandas apply
tqdm.pandas()

c:\Users\agust\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\agust\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [16]:
#Rows and columns df display

#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

#pd.reset_option('display.max_columns')
#pd.reset_option('display.max_rows')

### Processing of the .DOCX document inside .ZIP files

How documents look like

In [17]:

# Load the document
doc = Document('nytdoc1/WHO GETS WHAT.DOCX')

# Print the text in each paragraph
for paragraph in doc.paragraphs:
    print(paragraph.text)





WHO GETS WHAT
The New York Times
June 8, 1980, Sunday, Late City Final Edition


Copyright 1980 The New York Times Company
Section: Section 7; Page 13, Column 3; Book Review Desk; Review
Length: 1154 words
Byline: By MARC F. PLATTNER; Marc F. Plattner is on the staff of the Twentieth Century Fund and is the author of ''Rousseau's State of Nature.''
Body


THE ZERO-SUM SOCIETY Distribution and the Possibilities for Economic Change.  By Lester C. Thurow.  230 pp. New York: Basic Books. $12.95.
A NUMBER of recent magazine articles have called attention to the growing predominance of more conservative views among the leading economists of the younger generation. Lester Thurow, professor of economics and management at M.I.T., remains a notable exception to this trend. Widely re-garded as the most talented of the younger liberal economists, Mr. Thurow has also demonstrated considerable gifts at writing for a popular audience, as well as a willingness to address an impressive range of eth

In [18]:
folder_path = "nytdoc1"

# List to store Document objects
documents = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a .docx file
    if filename.endswith(".DOCX"):
        # Load the document and add it to the list
        doc_path = os.path.join(folder_path, filename)
        doc = Document(doc_path)
        documents.append(doc)

Check for standard paragraph positions

In [19]:
doc = Document('September Scenes at N.Y.U. _ Picnickers and Picket Lines.DOCX')

In [20]:
#Change index to check for consistency in paragraph order
copyright_pattern = r"Copyright\s+(\d{4})"
for i, paragraph in enumerate(doc.paragraphs):
    #if paragraph.text:
    #    print(f"Par {i}")
    #    print(paragraph.text)
    if len(paragraph.text.split()) == 1:  # Split text by whitespace and check if only one word is present
            print(paragraph.text)

    # if i == 7:
    #     print(f"Par {i}")
    #     print(paragraph.text)
    #     match = re.search(copyright_pattern, paragraph.text, re.IGNORECASE)
    #     year = int(match.group(1))
    #     print(match)
    #     print(year)


Body
Correction

Graphic


Along time, 3 types of dates are given, handle them

In [21]:
def process_date_format(text):
    """Process the different date formats and store them in the data dictionary."""
    #date_type == 1: January 27, 1986, Monday, Late City Final Edition
    #date_type == 2: January 5, 2014 Sunday
    #date_type == 3: July 8, 2021 Thursday 18:12 EST

    data = {}
    
    # Type 3: Date with time and timezone (e.g., "July 8, 2021 Thursday 18:12 EST")
    type_3_pattern = r"([A-Za-z]+ \d{1,2}, \d{4}) (\w+) (\d{1,2}:\d{2}) (\w+)"
    match_3 = re.match(type_3_pattern, text)
    if match_3:
        date_str = match_3.group(1) + ' ' + match_3.group(3)  # Combine date and time
        date_obj = datetime.strptime(date_str, '%B %d, %Y %H:%M')
        data["date"] = date_obj
        data["date_type"] = 3  # Mark as type 3 (with time and timezone)
        data["weekday"] = match_3.group(2)  # Extract the weekday
        data["timezone"] = match_3.group(4)  # Extract the timezone (e.g., "EST")
        return data

    # Type 1: Full date with weekday and extra information (e.g., "January 27, 1986, Monday, Late City Final Edition")
    type_1_pattern = r"([A-Za-z]+ \d{1,2}, \d{4}), (\w+), (.+)"
    match_1 = re.match(type_1_pattern, text)
    if match_1:
        date_str = match_1.group(1)
        weekday = match_1.group(2)
        issue = match_1.group(3)
        date_obj = datetime.strptime(date_str, '%B %d, %Y')
        data["date"] = date_obj
        data["date_type"] = 1  # Mark as type 1 (with weekday and extra info)
        data["weekday"] = weekday  # Extract the weekday
        data["issue"] = issue  # Extract the trailing text (e.g., "Late City Final Edition")
        return data
    
    # Type 2: Date with weekday but no extra information (e.g., "January 5, 2014 Sunday")
    type_2_pattern = r"([A-Za-z]+ \d{1,2}, \d{4}) (\w+)"
    match_2 = re.match(type_2_pattern, text)
    if match_2:
        date_str = match_2.group(1)
        date_obj = datetime.strptime(date_str, '%B %d, %Y')
        data["date"] = date_obj
        data["date_type"] = 2  # Mark as type 2 (with weekday)
        data["weekday"] = match_2.group(2)  # Extract the weekday
        return data

    # If no match found, return data without modification
    return data

In [22]:
text = "July 8, 2021 Thursday 18:12 EST"
result = process_date_format(text)
print(result)

text = "January 27, 1986, Monday, Late City Final Edition"
result = process_date_format(text)
print(result)

text = "January 5, 2014 Sunday"
result = process_date_format(text)
print(result)

{'date': datetime.datetime(2021, 7, 8, 18, 12), 'date_type': 3, 'weekday': 'Thursday', 'timezone': 'EST'}
{'date': datetime.datetime(1986, 1, 27, 0, 0), 'date_type': 1, 'weekday': 'Monday', 'issue': 'Late City Final Edition'}
{'date': datetime.datetime(2014, 1, 5, 0, 0), 'date_type': 2, 'weekday': 'Sunday'}


.DOCX parsing function

In [23]:
def extract_info_from_docx(doc, folder_name, four_word_list):
    '''# title: 3rd paragraph of .DOCX
        # outlet: 4th par

        # See process_date_format() function for 5th par processing of date
        # date_d-m-Y: 5th par, strftime("%d-%m-%Y") format
        # date_datetime: 5th par, datetime format
        # weekday: 5th par
        # issue: 5th par
        # exact_date: when False only the year is accurate (date set to 1st January)
        # date_type: date_type == 1: January 27, 1986, Monday, Late City Final Edition
        #            date_type == 2: January 5, 2014 Sunday
        #            date_type == 3: July 8, 2021 Thursday 18:12 EST

        # byline: par starting with	"Byline:"
        # dateline: par starting with "Dateline:", sometimes can differ from release date (author may have send it before publication date)	
        # section: par starting with "Section:"
        #         "Section 7; Page 13, Column 3; Book Review Desk; Review"
        #         "Section B; Column 0; Business/Financial Desk; Pg. 1; ECONOMIC SCENE"
        # length_words: par starting with "Length:", length in words
        # highlight: par starting with "Highlight:"
        # body: text below "Body" par up to ("Graphic", "End of document", "Correction", "Related Articles")
        # load_date: par starting with "Load-Date:"
        # 
        # Relevant flags for content analysis
        # summary_article: articles with many topics
        # transcript: article is a transcript (stayed in title)
        # to_editor: article is a letter to the editor

        # folder_name: name of .ZIP folder downloaded from NexisUni'''

    # Initialize an empty dictionary to store extracted data
    data = {
        "title": "",
        "outlet": "",
        "date_d-m-Y": "",
        "date_datetime": "",
        "weekday": "",
        "issue": "",
        "exact_date": True,
        "date_type": "",
        "byline": "",
        "dateline": "",
        "section": "",
        "length_words": "",
        "highlight": "",
        "body": "",
        "load_date": "",
        "has_body": False,
        "summary_article": False,
        "transcript": False,
        "wrong_retrieval": False,
        "to_editor": False,
        "folder_name": folder_name
    }
    
    # Flags for parsing
    body_text = []
    in_body = False  # To track when we’re in the body section
    date_extracted = False

    # Lists and regex
        
    #Flags
    summary_article_titles = ("The Playlist", "Inside the times", "Business digest", "The Digest", "New York Today",
                            "the week ahead", "news summary", "in summary", "business diary", "letters", "corrections", "the listings", 
                            "world briefing", "DealBook Briefing", "monday briefing", "tuesday briefing", "wednesday briefing",
                            "thursday briefing", "friday briefing",  "saturday briefing", "sunday briefing","Morning Briefing",
                            "DealBook Newsletter", "Morning Agenda", "Daily Clip Report")
    
    summary_article_innertitles = ("the latest news", "the speed read", "good morning.","Dear Diary:")
    
    transcript_title = "transcript"
    wrong_retrieval_article_titles = "Best Sellers:" #unknown reason

        #End of body titles
    end_of_body_subtitles = ("Graphic", "End of Document", "Related Articles")
    
        #Dates
    months = ["January", "February", "March", "April", "May", "June", "July", \
                  "August", "September", "October", "November", "December"]
    
    copyright_pattern = r"Copyright\s+(\d{4})" #for date retrieval
    
    # Loop through paragraphs and extract data
    for idx, paragraph in enumerate(doc.paragraphs):
        text = paragraph.text.strip()

        # Check if the paragraph has just 4 words (assesment of usual inner titles)
        if len(text.split()) < 5:  
            four_word_list.append(text)
        
        # Extract title
        if idx == 3:         
            data["title"] = text

            # Flag summary articles
            if any(keyword.lower() in data["title"].lower() for keyword in summary_article_titles):
                data["summary_article"] = True  # Set to True if it matches any of the keywords

            # Flag transcripts
            if "transcript" in data["title"].lower():
                data["transcript"] = True  # Set to True if it matches any of the keywords

            if "Best Sellers:".lower() in data["title"].lower():
                data["wrong_retrieval"] = True  # Set to True if it matches any of the keywords
        
        # Extract outlet
        if idx == 4:
            data["outlet"] = text
        
        # Extract date
        if idx < 7 and any(text.startswith(month) for month in months):
            date_extracted = True
            date_info = process_date_format(text)
            
            if "date" in date_info:
                data["date_d-m-Y"] = date_info["date"].strftime("%d-%m-%Y")
                data["date_datetime"] = date_info["date"]
            if "weekday" in date_info:
                data["weekday"] = date_info["weekday"]
            if "issue" in date_info:
                data["issue"] = date_info["issue"]
            if "date_type" in date_info:
                data["date_type"] = date_info["date_type"]

        if idx < 8 and not date_extracted and re.search(copyright_pattern, text, re.IGNORECASE):
            # Find the year in the copyright text
            match = re.search(copyright_pattern, text, re.IGNORECASE)
            if match:
                # Extract the year and create a datetime object
                year = int(match.group(1))
                if year == 1988:
                    print(data["title"])
                data["date_datetime"] = datetime(year, 1, 1)  # Set to January 1st of that year
                data["date_d-m-Y"] = data["date_datetime"].strftime("%d-%m-%Y")
                data["exact_date"] = False
                date_extracted = True

        # Extract byline
        if text.startswith("Byline:"):
            byline_text = text.replace("Byline:", "").strip()
            # Remove "By " or "by " if it starts with either
            if byline_text.startswith("By ") or byline_text.startswith("by "):
                byline_text = byline_text[3:].strip()  # Remove the first 3 characters
            data["byline"] = byline_text
        
        # Extract dateline
        if text.startswith("Dateline:"):
            data["dateline"] = text.replace("Dateline:", "").strip()
        
        # Extract section
        if text.startswith("Section:"):
            data["section"] = text.replace("Section:", "").strip()
        
        # Extract length
        if text.startswith("Length:"):
            length_text = text.replace("Length:", "").strip()
            
            # Match pattern for "[number] words"
            length_match = re.match(r"(\d+)\s+words", length_text)
            if length_match:
                # Extract the number of words
                data["length_words"] = length_match.group(1)
            else:
                # Otherwise, keep the full text
                data["length_words"] = length_text
        
        if text.startswith("Highlight:"):
            data["highlight"] = text.replace("Highlight:", "").strip()

        if text.startswith("Load-Date:"):
                data["load_date"] = text.replace("Load-Date:", "").strip()
        
        # Start collecting body text after "Body:" until end markers
        if text == "Body":
            in_body = True  # Start collecting body text
            data["has_body"] = True

        if any(text.startswith(prefix) for prefix in end_of_body_subtitles):
            in_body = False
        
        if in_body:
            
            # Flag "to the editor" cases
            if "to the editor:" in text.lower():
                data["to_editor"] = True

            # Stop collecting body text if we encounter end markers
            
            if any(text.startswith(prefix) for prefix in summary_article_innertitles):
                data["summary_article"] = True
                
            body_text.append(text)
    
    # Join collected body text and add to data dictionary
    data["body"] = "\n".join(body_text).strip("\n")[4:] #reunite with newline, exceptuate "Body" beginning
    
    return data, four_word_list


.DOCX are in many .ZIP folders, extract and process

In [24]:
def extract_docx_from_zip(zip_path):
    """Extract .docx files from a ZIP archive."""
    docx_files = []
    
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Get list of all file names in the ZIP archive
        for file_name in zip_ref.namelist():
            if file_name.endswith('.DOCX'):
                # Read the .docx file into memory
                with zip_ref.open(file_name) as docx_file:
                    docx_files.append(BytesIO(docx_file.read()))
    print(docx_files)
    return docx_files

In [25]:
def process_zip_folders(folder_path, four_word_list =[]):
    """Process all ZIP files in a folder and extract data from .docx files."""
    all_data = []
    four_word_list = []

    # Get the name of the last folder (e.g., "nyt80_1")
    folder_name = os.path.basename(folder_path)
    
    # Get a list of all ZIP files in the folder
    zip_files = [file_name for file_name in os.listdir(folder_path) if file_name.endswith('.ZIP')]
    
    # Sort the ZIP files by the numeric part of the filenames (e.g., '_1.ZIP', '_2.ZIP', ...)
    zip_files.sort(key=lambda x: int(re.search(r'_(\d+)\.ZIP', x).group(1)))

    # Loop through all the sorted ZIP files
    for file_name in zip_files:
        zip_path = os.path.join(folder_path, file_name)
        print(f"Processing: {zip_path}")
        
        # Assuming extract_docx_from_zip is defined elsewhere
        docx_files = extract_docx_from_zip(zip_path)
            
        # For each .docx file in the ZIP archive, extract data
        for docx_bytes in docx_files:
                doc = Document(docx_bytes)
                doc_data, four_word_list = extract_info_from_docx(doc, file_name, four_word_list)
                all_data.append(doc_data)
    
    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(all_data)
    return df, four_word_list

In [26]:
folders_path = "all_zip"
df, four_word = process_zip_folders(folders_path)

Processing: all_zip\nyt80_1.ZIP
[<_io.BytesIO object at 0x000002329B574090>, <_io.BytesIO object at 0x000002329B56B310>, <_io.BytesIO object at 0x0000023284E3D630>, <_io.BytesIO object at 0x0000023284E3DEA0>, <_io.BytesIO object at 0x0000023284E3D6D0>, <_io.BytesIO object at 0x0000023284E3D680>, <_io.BytesIO object at 0x0000023284E3D9A0>, <_io.BytesIO object at 0x0000023284E3D9F0>, <_io.BytesIO object at 0x0000023284E3D770>, <_io.BytesIO object at 0x0000023299405540>, <_io.BytesIO object at 0x0000023284E3D450>, <_io.BytesIO object at 0x0000023285E6E7C0>, <_io.BytesIO object at 0x0000023284E3DA90>, <_io.BytesIO object at 0x0000023299405AE0>, <_io.BytesIO object at 0x0000023284E3D3B0>, <_io.BytesIO object at 0x0000023284E3D950>, <_io.BytesIO object at 0x0000023284E3DBD0>, <_io.BytesIO object at 0x0000023284E3DB30>, <_io.BytesIO object at 0x0000023284E3DAE0>, <_io.BytesIO object at 0x0000023284E3DA40>, <_io.BytesIO object at 0x0000023284E3DCC0>, <_io.BytesIO object at 0x0000023284E3DC20>,

C:\Users\agust\AppData\Local\Temp\ipykernel_39088\3719675998.py:30: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  df = pd.DataFrame(all_data)


Metadata adds non-informative line "Job Number", handle it

In [27]:
#df row
#501	Job Number: = 238597316 (in title, first row)
nyt_parsed_data_df = df[~df["title"].str.startswith("Job Number")].reset_index(drop=True)

Date NAs

In [28]:
nyt_parsed_data_df[nyt_parsed_data_df['date_datetime'].isna()]

,title,outlet,date_d-m-Y,date_datetime,weekday,issue,exact_date,date_type,byline,dateline,...,length_words,highlight,body,load_date,has_body,summary_article,transcript,wrong_retrieval,to_editor,folder_name
692,September Scenes at N.Y.U.: Picnickers and Pic...,The New York Times,,NaT,,,True,,JANE GROSS,,...,1326,,\n\n\nIn a shady glade in Washington Square Pa...,,True,False,False,False,False,nyt80_2.ZIP


Hard code the one case that is not being handle with safe condition

In [29]:
condition = nyt_parsed_data_df['title'].str.lower().str.startswith('september')

# Check if the condition matches index 692 and update date_datetime
if condition.loc[692]:  # If the condition is True at index 692
    print("692 index is 'September Scenes' article")
    nyt_parsed_data_df.at[692, 'date_datetime'] = datetime(1988, 1, 1)  # Set date to 1988-01-01
    nyt_parsed_data_df.at[692, 'date_d-m-Y'] = nyt_parsed_data_df.at[692, 'date_datetime'].strftime("%d-%m-%Y")  # Update date_d-m-Y
    nyt_parsed_data_df.at[692, 'exact_date'] = False

692 index is 'September Scenes' article


Export first df output

In [30]:
#DF with all text extractions, metadata handling and one-row date error
nyt_parsed_data_df.to_csv('python_objects/nyt_parsed_data_df.csv', index=False)

In [31]:
#List as pickle
with open('python_objects/nyt_four_word_list.pkl', 'wb') as f:
    pickle.dump(four_word, f)

Import first df output

In [32]:
nyt_parsed_data_df = pd.read_csv('python_objects/nyt_parsed_data_df.csv')

Create column cleaned of middle \n and \r markers from title and body

In [33]:
def clean_html_in_data(df, body):
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Clean the 'title' column and create a '_clean_html' column for it
        if 'title' in df.columns:
            cleaned_title = re.sub(r"[\n\r\xa0]+", " ", row['title'])
            df.at[index, 'title_clean_html'] = cleaned_title  # Create new column with cleaned title
        
        if body:
            # Clean the 'body' column and create a '_clean_html' column for it
            if 'body' in df.columns:
                body_content = row['body']
                if isinstance(body_content, str):  # Ensure it's a string
                    cleaned_body = re.sub(r"[\n\r\xa0]+", " ", body_content)
                    df.at[index, 'body_clean_html'] = cleaned_body  # Create new column with cleaned body
                else:
                    df.at[index, 'body_clean_html'] = None  # Handle NaN or non-string cases
    
    return df

In [34]:
nyt_html_cleaned = clean_html_in_data(nyt_parsed_data_df, body = True)

Title + Body column

In [35]:
# Clean the combined string to remove any remaining HTML tags
nyt_html_cleaned['title_body_no_html'] = nyt_html_cleaned['title_clean_html'] + ' ' + nyt_html_cleaned['body_clean_html']

Export

In [36]:
nyt_html_cleaned['id_'] = nyt_html_cleaned.index

In [37]:
# DF with 'title_clean_html' and 'body_clean_html' with r"[\n\r\xa0]+" markups extracted
nyt_html_cleaned.to_csv('python_objects/nyt_html_cleaned.csv', index=False)

In [38]:
#Articles number before extracting duplicates and summary articles
len(nyt_html_cleaned)

14538